In [158]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Read from json file and convert to pandas dataframe
phone_call_df = pd.read_json('mobilecomputing-7b934-default-rtdb-export.json')

# Extract a dataframe where the userId is the same as the column name
phone_call_df = phone_call_df.transpose()
phone_call_df = phone_call_df.reset_index()
phone_call_df = phone_call_df.rename(columns={'index': 'userId'})

# ====== Data cleaning ======

# Remove userId Agata, bob, chris, rasmus and sarah as they are not part of the experiment

phone_call_df = phone_call_df[phone_call_df['userId'] != 'Agata']
phone_call_df = phone_call_df[phone_call_df['userId'] != 'bob']
# phone_call_df = phone_call_df[phone_call_df['userId'] != 'chris']
# phone_call_df = phone_call_df[phone_call_df['userId'] != 'rasmus']
# phone_call_df = phone_call_df[phone_call_df['userId'] != 'sarah']

# Remove headers
phone_call_arr = phone_call_df[:].values
phone_call_df = pd.DataFrame(phone_call_arr)

# Get a dataframe based on  the userId
def get_user_df(user_id):
    user_df = phone_call_df[phone_call_df[0] == user_id]
    user_df = user_df.dropna(axis=1, how='all')

    # Remove the userId column
    user_df = user_df.drop([0], axis=1)
    user_df = user_df.transpose()
    user_df = user_df.reset_index()
    user_df = user_df.drop(['index'], axis=1)
    user_df = user_df.rename(columns={list(user_df)[0]: 'gestureLog'})

    user_df = pd.DataFrame.from_records(user_df['gestureLog'])

    return user_df

# For each user, get a dataframe and append it to a dataframe
def get_all_users_df():
    all_users_df = pd.DataFrame()
    for user_id in phone_call_df[0]:
        user_df = get_user_df(user_id)
        all_users_df = pd.concat([all_users_df, user_df], ignore_index=True)
    all_users_df = all_users_df.reset_index()
    all_users_df = all_users_df.drop(['index'], axis=1)
    return all_users_df

data_df = get_all_users_df()

# Merge sarah experiments into one
data_df['userId'].replace({'sarah t2': 'sarah T2'}, inplace=True)




In [159]:
# Group data by userId
data_by_user = data_df.groupby('userId')

# Get how many times the actualGesture is different than the expectedGesture
def get_wrong_gestures(user_df):
    wrong_gestures = 0
    for index, row in user_df.iterrows():
        if row['actualGesture'] != row['expectedGesture']:
            wrong_gestures += 1
    return wrong_gestures

print(data_by_user.apply(get_wrong_gestures))
print(data_by_user.count())


userId
Angelos       0
Chris T2      4
Falcon       10
Gabriel       0
Katrine       1
Lotte         4
chris         0
rasmus        2
rasmus T2     7
sarah        30
sarah T2      9
dtype: int64
           actualGesture  contextId  expectedGesture  responseTime  \
userId                                                               
Angelos               52         52               52            52   
Chris T2              23         23               23            23   
Falcon                63         63               63            63   
Gabriel               48         48               48            48   
Katrine               47         47               47            47   
Lotte                 48         48               48            48   
chris                 48         48               48            48   
rasmus                39         39               39            39   
rasmus T2             30         30               30            30   
sarah                 82         8

In [173]:
data_df[data_df['userId'] == 'rasmus']

,actualGesture,contextId,expectedGesture,responseTime,timeStampAlarmSop,timeStampAlarmStart,userId
329,SpinHalf,Sit,SpinHalf,2413,1685992204512,1685992202099,rasmus
330,SpinHalf,Sit,SpinHalf,3609,1685992213981,1685992210372,rasmus
331,SpinHalf,Sit,SpinHalf,2409,1685992223475,1685992221066,rasmus
332,SpinHalf,Sit,SpinHalf,2210,1685992231150,1685992228940,rasmus
333,RotateHalf,Sit,RotateHalf,2013,1685992265299,1685992263286,rasmus
334,RotateHalf,Sit,RotateHalf,1813,1685992273037,1685992271224,rasmus
335,RotateHalf,Sit,RotateHalf,1812,1685992281729,1685992279917,rasmus
336,RotateHalf,Sit,RotateHalf,1614,1685992290075,1685992288461,rasmus
337,FlipHalf,Sit,FlipHalf,1811,1685992318088,1685992316277,rasmus
338,FlipHalf,Sit,FlipHalf,2213,1685992326693,1685992324480,rasmus


In [136]:
data_df[data_df['userId'] == 'sarah']['contextId'].unique()

array(['Sit', 'Distracted (Reading out loud)', 'Distracted (Typing)'],
      dtype=object)